In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from itertools import product
import os, sys

In [2]:
USA=False

In [3]:
if USA==True:
    result_root = "/mnt/CIL_integration/USA_SCC_rff2"
    sectors=['mortality_USA', 'agriculture_USA', 'energy_USA', 'labor_USA']
else:
    result_root = "/mnt/CIL_integration/rff2"
    sectors=['CAMEL', 'mortality', 'agriculture', 'energy', 'labor']
recipes=['equity',]
discs=['constant', 'euler_ramsey']
kinds=['p50', 'p75', 'p99.99', 'p0.01', 'zeroes']
masks=['epa_mask', 'unmasked']
eta_rhos = {
    2.0 : 0.0,
    1.016010255: 9.149608e-05,
    1.244459066: 0.00197263997,
    1.421158116: 0.00461878399,
    1.567899395: 0.00770271076,
}

In [4]:
def get_rff_id(sector,
               mask,
               recipe,
               disc,
               eta,
               rho,
               kind,
              ):

    if mask in ['epa_mask', 'unmasked']:
        results = f"{result_root}/{sector}/2020/unmasked_None/"
    elif mask=='gdppc_mask':
        results=f'{result_root}/{sector}/2020/gdppc_emissions_q0.01_q0.99'

    outdir=f"{result_root}/{sector}/2020/unmasked_None/"
    
    sccs = xr.open_dataset(
            f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_sccs.nc4"
        ).sel(weitzman_parameter='0.5', fair_aggregation='uncollapsed', gas='CO2_Fossil', drop=True).uncollapsed_sccs
        
    # runid rff_sp-simulation crosswalk
    cw = xr.open_dataset('/shares/gcp/integration/rff2/rffsp_fair_sequence.nc')
    
    if mask=='epa_mask':
        rffsp_values = pd.read_csv("/shares/gcp/integration/rff/rffsp_trials.csv").rffsp_id.unique()
        runid_idx = cw.where(cw.rff_sp.isin(rffsp_values), drop=True)
        sccs = sccs.sel(runid=runid_idx.runid)
    
    if kind=='zeroes':
        rff_ids = sccs.where((sccs >-1) & (sccs<1), drop=True)
    elif kind=='p0.01':
        cutoff = sccs.quantile(0.0001, 'runid')
        rff_ids = sccs.where(sccs < cutoff, drop=True)
    elif kind=='p50':
        cutoff_a = sccs.quantile(0.499, 'runid')
        cutoff_b = sccs.quantile(0.501, 'runid')
        rff_ids = sccs.where((sccs > cutoff_a) & (sccs < cutoff_b), drop=True)
    elif kind=='p75':
        cutoff_a = sccs.quantile(0.749, 'runid')
        cutoff_b = sccs.quantile(0.751, 'runid')
        rff_ids = sccs.where((sccs > cutoff_a) & (sccs < cutoff_b), drop=True)
    elif kind=='p99.99':
        cutoff = sccs.quantile(0.9999, 'runid')
        rff_ids = sccs.where(sccs > cutoff, drop=True)

    csv=pd.DataFrame(list(rff_ids['runid'].values), columns=['runid'])
    os.makedirs(f"{outdir}/runids/", exist_ok=True)
    csv.to_csv(f"{outdir}/runids/{recipe}_{disc}_eta{eta}_rho{rho}_{kind}_{mask}_runids.csv", index=False)

In [5]:
for sector, mask, recipe, disc, er, kind in product(sectors, masks, recipes, discs, eta_rhos.items(), kinds):
    # print(sector, mask, recipe, disc, er, kind)
    get_rff_id(sector,
               mask,
               recipe,
               disc,
               er[0],
               er[1],
               kind,
              )